Your boss is interested in knowing whether NO2 exports more electricity than it imports from 
Germany, and if this has changed over time. Therefore, she has asked you to create a bar plot 
that shows the annual sums of imports and exports of electricity in NO2. T
he graph should: 
- contain a single bar plot that shows the annual sums of exports and imports of electricity 
in NO2 from Nordlink for each year in the data. 

- be stored as a png file called “figure_task3.png”. 
What is your conclusion? Does NO2 import or export more electricity to Germany? Have the 
annual sums of exports and imports changed over the years? 

In [97]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Define data path and pattern
DATA_PATH = './data/PhysicalFlows_12.1.G'
pattern = f'{DATA_PATH}/*_PhysicalFlows_12.1.G.csv'

# Load and combine all CSV files
data = []
for file in glob.glob(pattern):
    df = pd.read_csv(file, sep='\t', parse_dates=['DateTime'])
    data.append(df)

df.groupby()


ValueError: No axis named NO2 for object type DataFrame